# Radiotherapy Project

$$
\begin{align*}
    \min \quad & \frac{w_{S_1}}{|S_1|}\sum_j\sum_\theta\sum_b\sum_c f(S_1)_j + \frac{w_{S_2}}{|S_2|}\sum_j\sum_\theta\sum_b\sum_c f(S_2)_j\\
    \textrm{s.t.} \quad & LB_T \leq f(T)_j \leq UB_T, & \forall j\in T\\
    & f(S_1)_j \leq UB_{S_1}, & \forall j\in S_1, \forall\theta \\
    & f(S_2)_j \leq UB_{S_2}, & \forall j\in S_2, \forall\theta \\
    & \theta \in \{1, 2\} \\
    & b \in \{1, 2, ..., 8\} \\
    & c \in \{1, 2, 3\}\\
    & t_{\theta,b,c} \in \mathbb{R^+} &~\text{}
\end{align*}
$$

where
$$f(S)_j = \sum_\theta\sum_b\sum_c D_{\theta,b,c}t_{\theta,b,c}$$

In [1]:
using JuMP, GLPK
using JuMP

In [ ]:
# 12 by 48 voxcels
dose_tumor = [1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 0.6 0.48 0.36 0.6 0.48 0.36 0.7 0.56 0.42 0.6 0.48 0.36 0.6 0.48 0.36 0.6 0.48 0.36 0.6 0.48 0.36 0.5 0.4 0.3;
0.9 0.72 0.54 0.95 0.76 0.57 0.85 0.68 0.51 0.85 0.68 0.51 0.9 0.72 0.54 0.95 0.76 0.57 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.65 0.52 0.39 0.65 0.52 0.39;
0.8 0.64 0.48 0.85 0.68 0.51 0.7 0.56 0.42 0.65 0.52 0.39 0.8 0.64 0.48 0.85 0.68 0.51 0.6 0.48 0.36 0.55 0.44 0.33 0.85 0.68 0.51 0.8 0.64 0.48 0.75 0.6 0.45 0.85 0.68 0.51 0.85 0.68 0.51 0.8 0.64 0.48 0.65 0.52 0.39 0.75 0.6 0.45;
0.95 0.76 0.57 0.9 0.72 0.54 0.85 0.68 0.51 0.95 0.76 0.57 0.95 0.76 0.57 0.9 0.72 0.54 0.75 0.6 0.45 0.85 0.68 0.51 0.7 0.56 0.42 0.75 0.6 0.45 0.6 0.48 0.36 0.55 0.44 0.33 0.7 0.56 0.42 0.75 0.6 0.45 0.5 0.4 0.3 0.45 0.36 0.27;
0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.75 0.6 0.45 0.75 0.6 0.45 0.8 0.64 0.48 0.85 0.68 0.51 0.75 0.6 0.45 0.75 0.6 0.45 0.8 0.64 0.48 0.85 0.68 0.51 0.65 0.52 0.39 0.65 0.52 0.39;
0.7 0.56 0.42 0.7 0.56 0.42 0.8 0.64 0.48 0.7 0.56 0.42 0.7 0.56 0.42 0.7 0.56 0.42 0.7 0.56 0.42 0.6 0.48 0.36 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54;
0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.9 0.72 0.54 0.7 0.56 0.42 0.7 0.56 0.42 0.8 0.64 0.48 0.7 0.56 0.42 0.7 0.56 0.42 0.7 0.56 0.42 0.7 0.56 0.42 0.6 0.48 0.36;
0.8 0.64 0.48 0.85 0.68 0.51 0.75 0.6 0.45 0.75 0.6 0.45 0.8 0.64 0.48 0.85 0.68 0.51 0.65 0.52 0.39 0.65 0.52 0.39 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.85 0.68 0.51 0.75 0.6 0.45 0.75 0.6 0.45;
0.7 0.56 0.42 0.75 0.6 0.45 0.6 0.48 0.36 0.55 0.44 0.33 0.7 0.56 0.42 0.75 0.6 0.45 0.5 0.4 0.3 0.45 0.36 0.27 0.95 0.76 0.57 0.9 0.72 0.54 0.85 0.68 0.51 0.95 0.76 0.57 0.95 0.76 0.57 0.9 0.72 0.54 0.75 0.6 0.45 0.85 0.68 0.51;
0.85 0.68 0.51 0.8 0.64 0.48 0.75 0.6 0.45 0.85 0.68 0.51 0.85 0.68 0.51 0.8 0.64 0.48 0.65 0.52 0.39 0.75 0.6 0.45 0.8 0.64 0.48 0.85 0.68 0.51 0.7 0.56 0.42 0.65 0.52 0.39 0.8 0.64 0.48 0.85 0.68 0.51 0.6 0.48 0.36 0.55 0.44 0.33;
0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.75 0.6 0.45 0.65 0.52 0.39 0.65 0.52 0.39 0.9 0.72 0.54 0.95 0.76 0.57 0.85 0.68 0.51 0.85 0.68 0.51 0.9 0.72 0.54 0.95 0.76 0.57 0.75 0.6 0.45 0.75 0.6 0.45;
0.6 0.48 0.36 0.6 0.48 0.36 0.7 0.56 0.42 0.6 0.48 0.36 0.6 0.48 0.36 0.6 0.48 0.36 0.6 0.48 0.36 0.5 0.4 0.3 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6 1.0 0.8 0.6;]

In [75]:
# 12 by 48 voxcels
dose_s1 = [0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.7 0.56 0.42 0.6 0.48 0.36 0.5 0.4 0.3 0.6 0.48 0.36 0.6 0.48 0.36 0.2 0.16 0.12 0.3 0.24 0.18 0.5 0.4 0.3 0.4 0.32 0.24 0.3 0.24 0.18 0.2 0.16 0.12 0.3 0.24 0.18 0.2 0.16 0.12;
0.4 0.32 0.24 0.55 0.44 0.33 0.55 0.44 0.33 0.55 0.44 0.33 0.5 0.4 0.3 0.45 0.36 0.27 0.35 0.28 0.21 0.35 0.28 0.21 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.55 0.44 0.33 0.45 0.36 0.27 0.35 0.28 0.21 0.35 0.28 0.21 0.35 0.28 0.21;
0.3 0.24 0.18 0.45 0.36 0.27 0.4 0.32 0.24 0.35 0.28 0.21 0.4 0.32 0.24 0.35 0.28 0.21 0.2 0.16 0.12 0.15 0.12 0.09 0.45 0.36 0.27 0.5 0.4 0.3 0.55 0.44 0.33 0.65 0.52 0.39 0.55 0.44 0.33 0.4 0.32 0.24 0.35 0.28 0.21 0.45 0.36 0.27;
0.45 0.36 0.27 0.5 0.4 0.3 0.55 0.44 0.33 0.65 0.52 0.39 0.55 0.44 0.33 0.4 0.32 0.24 0.35 0.28 0.21 0.45 0.36 0.27 0.3 0.24 0.18 0.45 0.36 0.27 0.4 0.32 0.24 0.35 0.28 0.21 0.4 0.32 0.24 0.35 0.28 0.21 0.2 0.16 0.12 0.15 0.12 0.09;
0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.55 0.44 0.33 0.45 0.36 0.27 0.35 0.28 0.21 0.35 0.28 0.21 0.35 0.28 0.21 0.4 0.32 0.24 0.55 0.44 0.33 0.55 0.44 0.33 0.55 0.44 0.33 0.5 0.4 0.3 0.45 0.36 0.27 0.35 0.28 0.21 0.35 0.28 0.21;
0.2 0.16 0.12 0.3 0.24 0.18 0.5 0.4 0.3 0.4 0.32 0.24 0.3 0.24 0.18 0.2 0.16 0.12 0.3 0.24 0.18 0.2 0.16 0.12 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.7 0.56 0.42 0.6 0.48 0.36 0.5 0.4 0.3 0.6 0.48 0.36 0.6 0.48 0.36;
0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.6 0.48 0.36 0.5 0.4 0.3 0.4 0.32 0.24 0.5 0.4 0.3 0.5 0.4 0.3 0.3 0.24 0.18 0.4 0.32 0.24 0.6 0.48 0.36 0.5 0.4 0.3 0.4 0.32 0.24 0.3 0.24 0.18 0.4 0.32 0.24 0.3 0.24 0.18;
0.3 0.24 0.18 0.45 0.36 0.27 0.45 0.36 0.27 0.45 0.36 0.27 0.4 0.32 0.24 0.35 0.28 0.21 0.25 0.2 0.15 0.25 0.2 0.15 0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.65 0.52 0.39 0.55 0.44 0.33 0.45 0.36 0.27 0.45 0.36 0.27 0.45 0.36 0.27;
0.2 0.16 0.12 0.35 0.28 0.21 0.3 0.24 0.18 0.25 0.2 0.15 0.3 0.24 0.18 0.25 0.2 0.15 0.1 0.08 0.06 0.05 0.04 0.03 0.55 0.44 0.33 0.6 0.48 0.36 0.65 0.52 0.39 0.75 0.6 0.45 0.65 0.52 0.39 0.5 0.4 0.3 0.45 0.36 0.27 0.55 0.44 0.33;
0.35 0.28 0.21 0.4 0.32 0.24 0.45 0.36 0.27 0.55 0.44 0.33 0.45 0.36 0.27 0.3 0.24 0.18 0.25 0.2 0.15 0.35 0.28 0.21 0.4 0.32 0.24 0.55 0.44 0.33 0.5 0.4 0.3 0.45 0.36 0.27 0.5 0.4 0.3 0.45 0.36 0.27 0.3 0.24 0.18 0.25 0.2 0.15;
0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.45 0.36 0.27 0.35 0.28 0.21 0.25 0.2 0.15 0.25 0.2 0.15 0.25 0.2 0.15 0.5 0.4 0.3 0.65 0.52 0.39 0.65 0.52 0.39 0.65 0.52 0.39 0.6 0.48 0.36 0.55 0.44 0.33 0.45 0.36 0.27 0.45 0.36 0.27;
0.1 0.08 0.06 0.2 0.16 0.12 0.4 0.32 0.24 0.3 0.24 0.18 0.2 0.16 0.12 0.1 0.08 0.06 0.2 0.16 0.12 0.1 0.08 0.06 0.6 0.48 0.36 0.7 0.56 0.42 0.8 0.64 0.48 0.8 0.64 0.48 0.7 0.56 0.42 0.6 0.48 0.36 0.7 0.56 0.42 0.7 0.56 0.42]

12×48 Array{Float64,2}:
 0.5   0.4   0.3   0.6   0.48  0.36  …  0.3   0.24  0.18  0.2   0.16  0.12
 0.4   0.32  0.24  0.55  0.44  0.33     0.35  0.28  0.21  0.35  0.28  0.21
 0.3   0.24  0.18  0.45  0.36  0.27     0.35  0.28  0.21  0.45  0.36  0.27
 0.45  0.36  0.27  0.5   0.4   0.3      0.2   0.16  0.12  0.15  0.12  0.09
 0.35  0.28  0.21  0.45  0.36  0.27     0.35  0.28  0.21  0.35  0.28  0.21
 0.2   0.16  0.12  0.3   0.24  0.18  …  0.6   0.48  0.36  0.6   0.48  0.36
 0.4   0.32  0.24  0.5   0.4   0.3      0.4   0.32  0.24  0.3   0.24  0.18
 0.3   0.24  0.18  0.45  0.36  0.27     0.45  0.36  0.27  0.45  0.36  0.27
 0.2   0.16  0.12  0.35  0.28  0.21     0.45  0.36  0.27  0.55  0.44  0.33
 0.35  0.28  0.21  0.4   0.32  0.24     0.3   0.24  0.18  0.25  0.2   0.15
 0.25  0.2   0.15  0.35  0.28  0.21  …  0.45  0.36  0.27  0.45  0.36  0.27
 0.1   0.08  0.06  0.2   0.16  0.12     0.7   0.56  0.42  0.7   0.56  0.42

In [47]:
# 36 by 48 voxcels
dose_s2 = [0.0 0.0 0.0 0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.1 0.08 0.06 0.0 0.0 0.0 0.2 0.16 0.12 0.2 0.16 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.08 0.06 0.2 0.16 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.08 0.06 0.1 0.08 0.06;
0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.1 0.08 0.06 0.3 0.24 0.18 0.3 0.24 0.18 0.05 0.04 0.03 0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.1 0.08 0.06 0.05 0.04 0.03 0.2 0.16 0.12 0.2 0.16 0.12;
0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.2 0.16 0.12 0.4 0.32 0.24 0.4 0.32 0.24 0.15 0.12 0.09 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.15 0.12 0.09 0.3 0.24 0.18 0.3 0.24 0.18;
0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.3 0.24 0.18 0.5 0.4 0.3 0.5 0.4 0.3 0.25 0.2 0.15 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.25 0.2 0.15 0.4 0.32 0.24 0.4 0.32 0.24;
0.05 0.04 0.03 0.15 0.12 0.09 0.25 0.2 0.15 0.35 0.28 0.21 0.15 0.12 0.09 0.05 0.04 0.03 0.25 0.2 0.15 0.25 0.2 0.15 0.0 0.0 0.0 0.05 0.04 0.03 0.15 0.12 0.09 0.25 0.2 0.15 0.05 0.04 0.03 0.0 0.0 0.0 0.15 0.12 0.09 0.15 0.12 0.09;
0.15 0.12 0.09 0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.25 0.2 0.15 0.15 0.12 0.09 0.35 0.28 0.21 0.35 0.28 0.21 0.1 0.08 0.06 0.15 0.12 0.09 0.25 0.2 0.15 0.35 0.28 0.21 0.15 0.12 0.09 0.1 0.08 0.06 0.25 0.2 0.15 0.25 0.2 0.15;
0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.35 0.28 0.21 0.25 0.2 0.15 0.45 0.36 0.27 0.45 0.36 0.27 0.2 0.16 0.12 0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.25 0.2 0.15 0.2 0.16 0.12 0.35 0.28 0.21 0.35 0.28 0.21;
0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.45 0.36 0.27 0.35 0.28 0.21 0.55 0.44 0.33 0.55 0.44 0.33 0.3 0.24 0.18 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.35 0.28 0.21 0.3 0.24 0.18 0.45 0.36 0.27 0.45 0.36 0.27;
0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.1 0.08 0.06 0.3 0.24 0.18 0.3 0.24 0.18 0.05 0.04 0.03 0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.1 0.08 0.06 0.05 0.04 0.03 0.2 0.16 0.12 0.2 0.16 0.12;
0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.2 0.16 0.12 0.4 0.32 0.24 0.4 0.32 0.24 0.15 0.12 0.09 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.15 0.12 0.09 0.3 0.24 0.18 0.3 0.24 0.18;
0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.3 0.24 0.18 0.5 0.4 0.3 0.5 0.4 0.3 0.25 0.2 0.15 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.25 0.2 0.15 0.4 0.32 0.24 0.4 0.32 0.24;
0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.4 0.32 0.24 0.6 0.48 0.36 0.6 0.48 0.36 0.35 0.28 0.21 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.35 0.28 0.21 0.5 0.4 0.3 0.5 0.4 0.3;
0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.1 0.08 0.06 0.3 0.24 0.18 0.3 0.24 0.18 0.05 0.04 0.03 0.1 0.08 0.06 0.2 0.16 0.12 0.3 0.24 0.18 0.1 0.08 0.06 0.05 0.04 0.03 0.2 0.16 0.12 0.2 0.16 0.12;
0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.2 0.16 0.12 0.4 0.32 0.24 0.4 0.32 0.24 0.15 0.12 0.09 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.15 0.12 0.09 0.3 0.24 0.18 0.3 0.24 0.18;
0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.3 0.24 0.18 0.5 0.4 0.3 0.5 0.4 0.3 0.25 0.2 0.15 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.25 0.2 0.15 0.4 0.32 0.24 0.4 0.32 0.24;
0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.4 0.32 0.24 0.6 0.48 0.36 0.6 0.48 0.36 0.35 0.28 0.21 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.35 0.28 0.21 0.5 0.4 0.3 0.5 0.4 0.3;
0.15 0.12 0.09 0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.25 0.2 0.15 0.15 0.12 0.09 0.35 0.28 0.21 0.35 0.28 0.21 0.1 0.08 0.06 0.15 0.12 0.09 0.25 0.2 0.15 0.35 0.28 0.21 0.15 0.12 0.09 0.1 0.08 0.06 0.25 0.2 0.15 0.25 0.2 0.15;
0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.35 0.28 0.21 0.25 0.2 0.15 0.45 0.36 0.27 0.45 0.36 0.27 0.2 0.16 0.12 0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.25 0.2 0.15 0.2 0.16 0.12 0.35 0.28 0.21 0.35 0.28 0.21;
0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.45 0.36 0.27 0.35 0.28 0.21 0.55 0.44 0.33 0.55 0.44 0.33 0.3 0.24 0.18 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.35 0.28 0.21 0.3 0.24 0.18 0.45 0.36 0.27 0.45 0.36 0.27;
0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.75 0.6 0.45 0.55 0.44 0.33 0.45 0.36 0.27 0.65 0.52 0.39 0.65 0.52 0.39 0.4 0.32 0.24 0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.45 0.36 0.27 0.4 0.32 0.24 0.55 0.44 0.33 0.55 0.44 0.33;
0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.2 0.16 0.12 0.4 0.32 0.24 0.4 0.32 0.24 0.15 0.12 0.09 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.15 0.12 0.09 0.3 0.24 0.18 0.3 0.24 0.18;
0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.3 0.24 0.18 0.5 0.4 0.3 0.5 0.4 0.3 0.25 0.2 0.15 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.25 0.2 0.15 0.4 0.32 0.24 0.4 0.32 0.24;
0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.4 0.32 0.24 0.6 0.48 0.36 0.6 0.48 0.36 0.35 0.28 0.21 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.35 0.28 0.21 0.5 0.4 0.3 0.5 0.4 0.3;
0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.8 0.64 0.48 0.6 0.48 0.36 0.5 0.4 0.3 0.7 0.56 0.42 0.7 0.56 0.42 0.45 0.36 0.27 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.45 0.36 0.27 0.6 0.48 0.36 0.6 0.48 0.36;
0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.2 0.16 0.12 0.4 0.32 0.24 0.4 0.32 0.24 0.15 0.12 0.09 0.2 0.16 0.12 0.3 0.24 0.18 0.4 0.32 0.24 0.2 0.16 0.12 0.15 0.12 0.09 0.3 0.24 0.18 0.3 0.24 0.18;
0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.3 0.24 0.18 0.5 0.4 0.3 0.5 0.4 0.3 0.25 0.2 0.15 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.25 0.2 0.15 0.4 0.32 0.24 0.4 0.32 0.24;
0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.4 0.32 0.24 0.6 0.48 0.36 0.6 0.48 0.36 0.35 0.28 0.21 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.35 0.28 0.21 0.5 0.4 0.3 0.5 0.4 0.3;
0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.8 0.64 0.48 0.6 0.48 0.36 0.5 0.4 0.3 0.7 0.56 0.42 0.7 0.56 0.42 0.45 0.36 0.27 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.45 0.36 0.27 0.6 0.48 0.36 0.6 0.48 0.36;
0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.35 0.28 0.21 0.25 0.2 0.15 0.45 0.36 0.27 0.45 0.36 0.27 0.2 0.16 0.12 0.25 0.2 0.15 0.35 0.28 0.21 0.45 0.36 0.27 0.25 0.2 0.15 0.2 0.16 0.12 0.35 0.28 0.21 0.35 0.28 0.21;
0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.45 0.36 0.27 0.35 0.28 0.21 0.55 0.44 0.33 0.55 0.44 0.33 0.3 0.24 0.18 0.35 0.28 0.21 0.45 0.36 0.27 0.55 0.44 0.33 0.35 0.28 0.21 0.3 0.24 0.18 0.45 0.36 0.27 0.45 0.36 0.27;
0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.75 0.6 0.45 0.55 0.44 0.33 0.45 0.36 0.27 0.65 0.52 0.39 0.65 0.52 0.39 0.4 0.32 0.24 0.45 0.36 0.27 0.55 0.44 0.33 0.65 0.52 0.39 0.45 0.36 0.27 0.4 0.32 0.24 0.55 0.44 0.33 0.55 0.44 0.33;
0.55 0.44 0.33 0.65 0.52 0.39 0.75 0.6 0.45 0.85 0.68 0.51 0.65 0.52 0.39 0.55 0.44 0.33 0.75 0.6 0.45 0.75 0.6 0.45 0.5 0.4 0.3 0.55 0.44 0.33 0.65 0.52 0.39 0.75 0.6 0.45 0.55 0.44 0.33 0.5 0.4 0.3 0.65 0.52 0.39 0.65 0.52 0.39;
0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.3 0.24 0.18 0.5 0.4 0.3 0.5 0.4 0.3 0.25 0.2 0.15 0.3 0.24 0.18 0.4 0.32 0.24 0.5 0.4 0.3 0.3 0.24 0.18 0.25 0.2 0.15 0.4 0.32 0.24 0.4 0.32 0.24;
0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.4 0.32 0.24 0.6 0.48 0.36 0.6 0.48 0.36 0.35 0.28 0.21 0.4 0.32 0.24 0.5 0.4 0.3 0.6 0.48 0.36 0.4 0.32 0.24 0.35 0.28 0.21 0.5 0.4 0.3 0.5 0.4 0.3;
0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.8 0.64 0.48 0.6 0.48 0.36 0.5 0.4 0.3 0.7 0.56 0.42 0.7 0.56 0.42 0.45 0.36 0.27 0.5 0.4 0.3 0.6 0.48 0.36 0.7 0.56 0.42 0.5 0.4 0.3 0.45 0.36 0.27 0.6 0.48 0.36 0.6 0.48 0.36;
0.6 0.48 0.36 0.7 0.56 0.42 0.8 0.64 0.48 0.9 0.72 0.54 0.7 0.56 0.42 0.6 0.48 0.36 0.8 0.64 0.48 0.8 0.64 0.48 0.55 0.44 0.33 0.6 0.48 0.36 0.7 0.56 0.42 0.8 0.64 0.48 0.6 0.48 0.36 0.55 0.44 0.33 0.7 0.56 0.42 0.7 0.56 0.42]

36×48 Array{Float64,2}:
 0.0   0.0   0.0   0.1   0.08  0.06  …  0.1   0.08  0.06  0.1   0.08  0.06
 0.1   0.08  0.06  0.2   0.16  0.12     0.2   0.16  0.12  0.2   0.16  0.12
 0.2   0.16  0.12  0.3   0.24  0.18     0.3   0.24  0.18  0.3   0.24  0.18
 0.3   0.24  0.18  0.4   0.32  0.24     0.4   0.32  0.24  0.4   0.32  0.24
 0.05  0.04  0.03  0.15  0.12  0.09     0.15  0.12  0.09  0.15  0.12  0.09
 0.15  0.12  0.09  0.25  0.2   0.15  …  0.25  0.2   0.15  0.25  0.2   0.15
 0.25  0.2   0.15  0.35  0.28  0.21     0.35  0.28  0.21  0.35  0.28  0.21
 0.35  0.28  0.21  0.45  0.36  0.27     0.45  0.36  0.27  0.45  0.36  0.27
 0.1   0.08  0.06  0.2   0.16  0.12     0.2   0.16  0.12  0.2   0.16  0.12
 0.2   0.16  0.12  0.3   0.24  0.18     0.3   0.24  0.18  0.3   0.24  0.18
 0.3   0.24  0.18  0.4   0.32  0.24  …  0.4   0.32  0.24  0.4   0.32  0.24
 0.4   0.32  0.24  0.5   0.4   0.3      0.5   0.4   0.3   0.5   0.4   0.3 
 0.1   0.08  0.06  0.2   0.16  0.12     0.2   0.16  0.12  0.2   0.16  0.12
 

In [71]:
m = Model(with_optimizer(GLPK.Optimizer))
VoxelNum_tumor = 12
VoxelNum_s1 = 12
VoxelNum_s2 = 36
T = 2*8*3

@variable(m, t[1:T] >= 0)

weight_s1 = 1
weight_s2 = 1

@objective(m, Min, 
    (weight_s1/VoxelNum_s1) * (sum(dose_s1 * t)) + 
    (weight_s2/VoxelNum_s2) * (sum(dose_s2 * t)))

# @objective(m, Min, 
#     (weight_s1/VoxelNum_s1) * (sum(dose_s1[j,i] * t[i] for i in 1:48)) + 
#     (weight_s2/VoxelNum_s2) * (sum(dose_s2[j,i] * t[i] for i in 1:48)))

lb_tumor = 10
ub_tumor = 15
ub_s1 = 5
ub_s2 = 5
@constraint(m, lb_const_tumor[j in 1:VoxelNum_tumor], lb_tumor <= sum(dose_tumor[j,i] * t[i] for i in 1:48) <= ub_tumor)
@constraint(m, lb_const_s1[j in 1:VoxelNum_s1], sum(dose_s1[j,i] * t[i] for i in 1:48) <= ub_s1)
@constraint(m, lb_const_s2[j in 1:VoxelNum_s2], sum(dose_s2[j,i] * t[i] for i in 1:48) <= ub_s2)

optimize!(m)

print("t = ",value.(t))

t = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8196721311475396, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.19672131147541, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]